### Libraries import

In [1]:
import pandas as pd # Dataframes management
from zipfile import ZipFile  # Files compressed management
import os # Files management along OS
import re

In [2]:
from os import mkdir
from os.path import exists

In [3]:
from selenium import webdriver # Webscrapping bot
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.firefox.options import Options

In [4]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

### Modules

In [5]:
from db import *

### Data import

In [6]:
# Specifying the name of the zip file
fileZIP = "/items_ordered_2years_V2.zip"
fileCSV = "/items_ordered_2years_V2.csv"

path = "../Inputs/Modificados - Atmira_Pharma_Visualization"
  
# Open the zip file in read mode
with ZipFile(f"{path}{fileZIP}", 'r') as zip: 
    # List all the contents of the zip file
    zip.printdir() 
  
    # Extract all files
    print('extraction...') 
    zip.extractall(path) 
    print('Done!')

#Import CSV to pandas
itemsOrdered = pd.read_csv(f"{path}{fileCSV}")
print("CSV imported to Pandas successfully")

# Remove uncompressed CSV file
os.remove(f"{path}{fileCSV}")
print("Original CSV removed to preserve repo health")

File Name                                             Modified             Size
items_ordered_2years_V2.csv                    2022-02-23 01:28:04    150372498
extraction...
Done!
CSV imported to Pandas successfully
Original CSV removed to preserve repo health


In [7]:
itemsOrdered.head(3)

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
0,562ba4ba9aaf2a1e926842deca19271d,2bca87c2ab10b07feef290a9be47f316,2017-01-01 00:20:33,6409.0,1.0,13.4204,17.17,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019
1,562ba4ba9aaf2a1e926842deca19271d,53e12ac4b579f0c4b1c4f77d50bf8792,2017-01-01 00:20:33,5133.0,1.0,18.5351,22.68,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019
2,562ba4ba9aaf2a1e926842deca19271d,d19d6df5cd6a1e741408b70aa2767bf4,2017-01-01 00:20:33,5125.0,1.0,18.5260,22.68,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019


In [8]:
itemsOrdered.shape

(321671, 11)

In [9]:
del zip

### Arreglos para facilitar el webscrapping

In [10]:
%%capture
'''
Temas Portugueses
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("4575-049", "4575-001")
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("1750", "1750-001")
'''

In [11]:

itemsOrdered.loc[itemsOrdered['zipcode'].eq('30139') & itemsOrdered['city'].eq('Murcia'), "city"] = "EL RAAL"
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("29039", "28039")
itemsOrdered.loc[itemsOrdered['zipcode'].eq('33195') & itemsOrdered['city'].eq('Oviedo '), "city"] = "Morente"
itemsOrdered["city"].replace(to_replace={'Cangas Del Narcea':"Cerezaliz"}, inplace=True)
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("O88oo", "08800")
itemsOrdered.loc[itemsOrdered['zipcode'].eq('43890') & itemsOrdered['city'].eq('TARRAGONA'), "city"] = "Hospitalet"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('07700') & itemsOrdered['city'].eq('MAHON'), "city"] = "Grau"
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("47021", "46021")
itemsOrdered.loc[itemsOrdered['zipcode'].eq('33405'), "city"] = "Raices Nuevo"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('29720') & itemsOrdered['city'].eq('MALAGA'), "city"] = "La Arana"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('08222'), "city"] = "Terrassa"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('03195'), "city"] = "Los Arenales Del Sol"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('08780'), "city"] = "Palleja"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('33405'), "city"] = "Raices Nuevo"
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("347007", "37007")
itemsOrdered.loc[itemsOrdered['zipcode'].eq('36194'), "city"] = "Perdecanai"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('29631'), "city"] = "Arroyo De La Miel"
itemsOrdered.loc[itemsOrdered['zipcode'].eq('27810'), "city"] = "Sancobade"
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("-39840", "39840")


In [12]:
itemsOrdered[itemsOrdered["city"]=="Lisboa"].head(2)

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
1900,1133bd26dc28c6ee87798d00ae7166b9,35672b13aa794e7d56f59327d1bf4af8,2017-01-01 23:26:29,3726.0,1.0,16.3320,21.78,8.0,68e3029766c319b18533206bed2ef83f,Lisboa,1750 341 Lisboa
1901,1133bd26dc28c6ee87798d00ae7166b9,3f58086a75e4e96c3c24f3ce45e4d9fa,2017-01-01 23:26:29,11628.0,1.0,6.4432,8.40,8.0,68e3029766c319b18533206bed2ef83f,Lisboa,1750 341 Lisboa


### USO DE WEBSCRAPPING PARA CÓDIGO POSTAL

Creación de Base de Datos para almacenamiento de resultados formateados del WebScrapping

In [13]:
def IntroDB():
    try:
        pathDB = "../databases/"
        nameDB= "scrapedZips.db"
        if exists(pathDB):
            print(f"Carpeta {pathDB} encontrada")
        else:
            mkdir(pathDB)
            print (f"Creada carpeta {pathDB}")
    except OSError:
        print (f"Creación de carpeta {pathDB} falló")
        
    if exists(f"{pathDB}{nameDB}"):
        print("Database existe")
        con, cur = SqlConnection(f"{pathDB}{nameDB}")
        return con, cur
    else:
        print("Database no existe")
        CreateCon(f"{pathDB}{nameDB}")
        con, cur = SqlConnection(f"{pathDB}{nameDB}")
        PrepareCon(con, cur, option="insert",
            values=("Test","Test","Test","Test"))
        return con, cur


In [14]:
con, cur = IntroDB()

Creada carpeta ../databases/
Database no existe
Database creado
Conexión NO establecida
Realizando reparaciones
Creando tabla...
OK
Reparación completada
Conexión establecida
Insertando datos...
OK


In [15]:
try:
    if GetThings(cur, selection="Country,Region,City,Zipcode", where=["ID", 1], limit=1)[0][0] == "Test":
        PrepareCon(con,cur,where=["ID",1],option="delete")
        cur.execute("UPDATE `sqlite_sequence` SET `seq` = 0 WHERE `name` = 'ZipsInfo';")
except IndexError:
    pass

'''
Check the next query if something perturbe your mind
SELECT * FROM `sqlite_sequence`;
'''

Borrando fila...
OK


'\nCheck the next query if something perturbe your mind\nSELECT * FROM `sqlite_sequence`;\n'

In [16]:
GetThings(cur, selection="Country,Region,City,Zipcode", where=["ID", 1], limit=1)

[]

A continuación se genera una lista compuesta de tuplas compuestas de la siguiente forma: ("Ciudad", "Zipcode")

In [17]:
rawDataZipcode = list(zip(itemsOrdered["city"].tolist(), itemsOrdered["zipcode"].tolist()))
rawDataZipcode = rawDataZipcode[:int(len(rawDataZipcode)/30)]

In [18]:
rawDataZipcode[0]

('Valencia', '46019')

**Funciones destacadas**

Función que "limpia" los nombres de ciudades para mejorar su emparejamiento automático

In [19]:
def CityCleaner(text):
    stopWordSpanish = set(stopwords.words('spanish'))
    wordTokens = word_tokenize(AcentosLimpiador(text.lower()).rstrip()) 
    filteredSentence = [element for element in wordTokens if not element in stopWordSpanish] 
    return filteredSentence
 

Función que limpia zipcodes

In [20]:
#Limpieza de zipcodes con RegEx
def num_guion(string):
    """ Get a string with the numbers and hyphens of another string
    
    Args:
        df: string used to extract the string with numbers abd hyphens

    Returns:
        df: the string with numbers and hyphens
    """
    aux = re.match("([\d-]+)", str(string))
    try:
        return str(aux.group())
    except:
        return string


itemsOrdered["zipcode"] = itemsOrdered["zipcode"].apply(lambda x: num_guion(x))

Función que limpia los acentos con el fin de homogeneizar

In [21]:
def AcentosLimpiador(text):
	acentos = {'ñ':'n','á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'à':'a'}
	for ele in acentos:
		if ele in text:
			text = text.replace(ele, acentos[ele])
	return text

Función que formatea los resultados del webscrapping de forma adecuada a los requerimientos necesarios

In [22]:
def zipCodeManipulation(city, zipcode, queryResult="", saved = False):
  
    # This conditional checks if the zipcode info was scrapped and stored succesfully
    if saved == False:

        listTestingZipcode = queryResult.split("\n")
        indexMatchRegex = list(map(lambda x: [(m.start(0), m.end(0)) for m in re.finditer(r"[a-z][A-Z0-9]", x)], listTestingZipcode))

        resultScrapClean = []
        for pas,resultScrap in enumerate(listTestingZipcode):
            for pos,ele in enumerate(indexMatchRegex[pas]):
                if len(indexMatchRegex[pas])==2:
                    if pos ==0:
                        txt = resultScrap[:ele[0]+1]+","+resultScrap[ele[1]-1:]
                    elif pos ==1:
                        txt = txt[:ele[0]+2]+","+txt[ele[1]:]
                        resultScrapClean.append(txt)
                elif len(indexMatchRegex[pas])==3:
                    if pos ==0:
                        txt = resultScrap[:ele[0]+1]+","+resultScrap[ele[1]-1:]
                    elif pos ==1:
                        txt = txt[:ele[0]+2]+","+txt[ele[1]:]
                    elif pos ==2:
                        txt = txt[:ele[0]+3]+","+txt[ele[1]+1:]
                        resultScrapClean.append(txt)

        resultScrapListed = [element.split(",") for element in resultScrapClean]
        resultScrapRearr = [(element[0], element[1], element[-2], element[-1]) for element in resultScrapListed]

        resultZip = []
        for element in resultScrapRearr:
            for element2  in CityCleaner(element[2]):
                if element2 in CityCleaner(city):
                    resultZip.append(element)
                    break
        try:
            resultZip = resultZip[0]
        except IndexError:
            resultZip = []
            for element in resultScrapRearr:
                for element2 in CityCleaner(element[2]):
                    for element3 in CityCleaner(city):
                        if element2 .__contains__(element3):
                            resultZip.append(element)
                            break
            try:
                resultZip = resultZip[0]
            except IndexError:
                resultZip = []
                for element in resultScrapRearr:
                    for element2 in CityCleaner(element[2]):
                        for city in CityCleaner(element3):
                            if element3.__contains__(element2):
                                resultZip.append(element)
                                break
                resultZip = resultZip[0]  

    elif saved == True:
        resultZip = GetThings(cur, selection="Country,Region,City,Zipcode", where=["Zipcode", zipcode], limit=1, simplify = True)

    if resultZip[3].startswith(" L'"):
        resultZip = [resultZip[0],resultZip[1],resultZip[2],resultZip[3][3:]]
    
    if len(resultZip[3]) >= 20:
        if resultZip[0]=="Portugal":
            resultZip = [resultZip[0],resultZip[1],resultZip[2],zipcode]

    i = (resultZip[3]+resultZip[3]).find(resultZip[3], 1, -1)
    return resultZip if i == -1 else [resultZip[0],resultZip[1],resultZip[2],resultZip[3][:i]]

**Webscrapping!**

In [31]:
# %%capture

try:
    opts = Options()
    opts.add_argument("--headless")
    driver = webdriver.Firefox(options=opts)
    
    driver.get("https://worldpostalcode.com/lookup")
    driver.set_page_load_timeout(5)

    for pos, element in enumerate(rawDataZipcode):

        if element[1]=="323903":
            PrepareCon(con, cur, values=["China", "Zhejiang", "Lishui", "323903"], option="insert", verbose=False)
            continue
        if GetThings(cur, selection="Zipcode", where=["Zipcode", element[1].rstrip()], limit=1, simplify=True) == None:
            print("Para scrapear", "Ciudad:", element[0], "Zipcode:", element[1], "Orden del dataframe: ", pos)
            insertZipcode =driver.find_element(By.ID,"search")
            insertZipcode.clear()
            insertZipcode.send_keys(element[1])
            clickButtonZipcode =driver.find_element(By.CLASS_NAME,"submit")
            clickButtonZipcode.click()
            driver.set_page_load_timeout(5)

            getGeoInfo = driver.find_element(By.CLASS_NAME,"search_units")
            saveGeoInfo = getGeoInfo.text
            zipCodeDef = zipCodeManipulation(element[0], element[1], queryResult=saveGeoInfo)
            print("Scrapeado: ", zipCodeDef)
            print("---------------------------------------------------")

        else:
            if pos in [ele[0]-1 for ele in GetThings(cur, selection="ID", where=["Zipcode", element[1].rstrip()])]:
                print("Sin procesar ", "City: ", element[0],  "Zipcode: ", element[1],  "Orden del dataframe: ", pos)
                print("---------------------------------------------------")
                continue
            else:
                print("Ya guardado en BBDD ", "City: ", element[0],  "Zipcode: ", element[1],  "Orden del dataframe: ", pos)
                zipCodeDef = zipCodeManipulation(city=element[0], zipcode=element[1], saved = True)
                print("Devuelto de query: ", zipCodeDef)
                print("---------------------------------------------------")

        PrepareCon(con, cur, values=zipCodeDef, option="insert", verbose=False)

    driver.close()
    con.close()

except IndexError:
    driver.close()
    raise Exception(f"El fallo se ha producido con ciudad: {element[0]}, zipcode: {element[1]}")

except NoSuchElementException:
    print(f"No encontrado nada en ciudad: {element[0]}, zipcode: {element[1]}")



Valencia 46019
Sin procesar  City:  Valencia Zipcode:  46019 Orden del dataframe:  0
---------------------------------------------------
Valencia 46019
Sin procesar  City:  Valencia Zipcode:  46019 Orden del dataframe:  1
---------------------------------------------------
Valencia 46019
Sin procesar  City:  Valencia Zipcode:  46019 Orden del dataframe:  2
---------------------------------------------------
Almeria 04009
Sin procesar  City:  Almeria Zipcode:  04009 Orden del dataframe:  3
---------------------------------------------------
Almeria 04009
Sin procesar  City:  Almeria Zipcode:  04009 Orden del dataframe:  4
---------------------------------------------------
Almeria 04009
Sin procesar  City:  Almeria Zipcode:  04009 Orden del dataframe:  5
---------------------------------------------------
Terrassa 08225
Sin procesar  City:  Terrassa Zipcode:  08225 Orden del dataframe:  6
---------------------------------------------------
Terrassa 08225
Sin procesar  City:  Terrassa Zi

Exception: El fallo se ha producido con ciudad: Zaragoza, zipcode: 50620

Añadidos manuales a la base de datos

In [ ]:
GetThings(cur, selection="ID,Country,Region,City,Zipcode", where=["Zipcode", "28915 "], limit=1, simplify=True)


(453, 'Spain', 'Madrid', 'Leganes', '28915')

**Comprobaciones del scrapping y formateo posterior realizado**

In [ ]:
tableMain = "ZipsInfo"

In [ ]:
cur.execute(f"SELECT (select count() from {tableMain}) as count FROM {tableMain} LIMIT 1").fetchone()[0]

3907

**Transformación del scrapeo formateado a dataframe de Pandas**

In [ ]:
sqlToList = cur.execute(f"SELECT Country,Region,City,Zipcode FROM {tableMain}").fetchall()


In [ ]:
df = pd.DataFrame(sqlToList, columns=["Country","Region","City","Zipcode"])
df

,Country,Region,City,Zipcode
0,Spain,Comunidad Valenciana,Valencia,46019
1,Spain,Comunidad Valenciana,Valencia,46019
2,Spain,Comunidad Valenciana,Valencia,46019
3,Spain,Andalucia,Almeria,04009
4,Spain,Andalucia,Almeria,04009
...,...,...,...,...
2145,Spain,Andalucia,Sevilla,41018
2146,Spain,Andalucia,Sevilla,41018
2147,Spain,Andalucia,Sevilla,41018
2148,Spain,Andalucia,Sevilla,41018


In [ ]:
GetThings(cur, selection="Zipcode", where=["Zipcode", "28915"], limit=1, simplify=True)

('28915',)

In [30]:
"28915 ".rstrip()

'28915'